# Incremental average

Incremental average

$$s_{n}=\alpha * s_{n-1} + (1-\alpha)*x_{n}$$

for $x_{n}$ numpy array

In [ ]:
mport numpy as np

In [4]:
class IncrementalAverage():
    def __init__(self, ratio, initial_value=0.):
        self.value = initial_value
        self.ratio = ratio
    
    def iterate(self, value):
        self.value *= 1-self.ratio
        self.value += self.ratio*value

class IncrementalStatistic():
    def __init__(self, ratio, initial_avg = 0., initial_sqs = 0.):
        self.avg = IncrementalAverage(ratio, initial_avg)
        self.avg_sqs = IncrementalAverage(ratio, initial_sqs)
    
    def iterate(self, value):
        self.avg.iterate(value)
        self.avg_sqs.iterate(np.square(value))
    
    def std(self):
        self.std = np.sqrt(self.avg_sqs.value - np.square(self.avg.value)) + 0.0001
        return self.std